In [30]:
import pandas as pd
import nltk
import re
import random
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import reuters, stopwords
from nltk.tokenize import word_tokenize
import string
from nltk import bigrams, FreqDist, ConditionalFreqDist
from nltk.stem import PorterStemmer, WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\praka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\praka\AppData\Roaming\nltk_data...


In [7]:
df = pd.read_csv('Coursera_reviews.csv')
print(df.shape)
df.head()

(1048309, 5)


,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4.0,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4.0,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4.0,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4.0,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4.0,google-cbrs-cpi-training


In [10]:
# working with a sample to start with, comment this out later
# df1 = df
df1 = df.iloc[0:100].copy()
df1.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4.0,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4.0,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4.0,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4.0,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4.0,google-cbrs-cpi-training


In [23]:
def preprocess_text(text):
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = ''.join([i for i in text if not i.isdigit()])

    # Remove special characters with space
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase and tokenize and remove stopwords
    words = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    return ' '.join(words)  # rejoin to make a string

df1['clean_reviews'] = df1['reviews'].apply(preprocess_text)
df1.head()


,reviews,reviewers,date_reviews,rating,course_id,clean_reviews
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4.0,google-cbrs-cpi-training,pretty dry able pass two complete watches im h...
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4.0,google-cbrs-cpi-training,would better experience video screen shots wou...
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4.0,google-cbrs-cpi-training,information perfect program little annoying wa...
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4.0,google-cbrs-cpi-training,grammatical mistakes test made double take bad
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4.0,google-cbrs-cpi-training,excellent course training provided detailed ea...


In [27]:
print('Sample change before and after cleaning')
print(df1.iloc[0:1]['reviews'])
print('\n')
print(df1.iloc[0:1]['clean_reviews'])

0    Pretty dry, but I was able to pass with just two complete watches so I'm happy about that.  As usual there were some questions on the final exam that were NO WHERE in the course, which is annoying but far better than many microsoft tests I have taken.  Never found the suplimental material that the course references... but who cares... i passed!
Name: reviews, dtype: object


0    pretty dry able pass two complete watches im happy usual questions final exam course annoying far better many microsoft tests taken never found suplimental material course references cares passed
Name: clean_reviews, dtype: object


In [31]:
ps = PorterStemmer()
lm = WordNetLemmatizer()

def apply_stemmer_lemmetizer(text):
    words = word_tokenize(text)  # tokenising given string
    stemmed_words = [ps.stem(word) for word in words]  # stemming given string
    lemmatised_words = [lm.lemmatize(word) for word in stemmed_words]
    return ' '.join(lemmatised_words)  # rejoin to make a string

df1['lemmatised_reviews'] = df1['clean_reviews'].apply(apply_stemmer_lemmetizer)
df1.head()

,reviews,reviewers,date_reviews,rating,course_id,clean_reviews,lemmatised_reviews
0,"Pretty dry, but I was able to pass with just two complete watches so I'm happy about that. As usual there were some questions on the final exam that were NO WHERE in the course, which is annoying but far better than many microsoft tests I have taken. Never found the suplimental material that the course references... but who cares... i passed!",By Robert S,"Feb 12, 2020",4.0,google-cbrs-cpi-training,pretty dry able pass two complete watches im happy usual questions final exam course annoying far better many microsoft tests taken never found suplimental material course references cares passed,pretti dri abl pas two complet watch im happi usual question final exam cours annoy far better mani microsoft test taken never found supliment materi cours refer care pas
1,would be a better experience if the video and screen shots would sho on the side of the text that the instructor is going thru so that user does not have to go all the way to beginning of text to be able to view any slides instructor is showing.,By Gabriel E R,"Sep 28, 2020",4.0,google-cbrs-cpi-training,would better experience video screen shots would sho side text instructor going thru user go way beginning text able view slides instructor showing,would better experi video screen shot would sho side text instructor go thru user go way begin text abl view slide instructor show
2,Information was perfect! The program itself was a little annoying. I had to wait 30 to 45 minutes after watching the videos to to take the quiz. Other than that the information was perfect and passed the test with no issues!,By Jacob D,"Apr 08, 2020",4.0,google-cbrs-cpi-training,information perfect program little annoying wait minutes watching videos take quiz information perfect passed test issues,inform perfect program littl annoy wait minut watch video take quiz inform perfect pas test issu
3,A few grammatical mistakes on test made me do a double take but all in all not bad.,By Dale B,"Feb 24, 2020",4.0,google-cbrs-cpi-training,grammatical mistakes test made double take bad,grammat mistak test made doubl take bad
4,Excellent course and the training provided was very detailed and easy to follow.,By Sean G,"Jun 18, 2020",4.0,google-cbrs-cpi-training,excellent course training provided detailed easy follow,excel cours train provid detail easi follow


In [32]:
print('Sample change before and after lemmitisation')
print(df1.iloc[0:1]['clean_reviews'])
print('\n')
print(df1.iloc[0:1]['lemmatised_reviews'])

Sample change before and after lemmitisation
0    pretty dry able pass two complete watches im happy usual questions final exam course annoying far better many microsoft tests taken never found suplimental material course references cares passed
Name: clean_reviews, dtype: object


0    pretti dri abl pas two complet watch im happi usual question final exam cours annoy far better mani microsoft test taken never found supliment materi cours refer care pas
Name: lemmatised_reviews, dtype: object


In [33]:
# uncomment to improve efficiency in large datasets
# df1.drop('clean_reviews', axis=1, inplace=True)